# Advanced Portfolio Optimization using cvxpy

## Install cvxpy and other libraries

In [2]:
import sys
!{sys.executable} -m pip install -r requirements.txt

    100% |████████████████████████████████| 880kB 9.3MB/s eta 0:00:01
    100% |████████████████████████████████| 12.2MB 2.1MB/s eta 0:00:01  2% |▋                               | 245kB 14.7MB/s eta 0:00:01    8% |██▊                             | 1.0MB 19.3MB/s eta 0:00:01    16% |█████▏                          | 2.0MB 19.8MB/s eta 0:00:01    35% |███████████▏                    | 4.3MB 23.4MB/s eta 0:00:01    44% |██████████████▎                 | 5.4MB 24.9MB/s eta 0:00:01    64% |████████████████████▋           | 7.8MB 23.9MB/s eta 0:00:01    72% |███████████████████████▍        | 8.9MB 9.9MB/s eta 0:00:01    89% |████████████████████████████▊   | 10.9MB 18.4MB/s eta 0:00:01
    100% |████████████████████████████████| 26.2MB 1.3MB/s eta 0:00:01   12% |███▉                            | 3.2MB 25.3MB/s eta 0:00:01    16% |█████▎                          | 4.3MB 24.7MB/s eta 0:00:01    20% |██████▋                         | 5.4MB 23.5MB/s eta 0:00:01    29% |█████████▍                

## Imports

In [8]:
import cvxpy as cvx
import numpy as np
import quiz_tests_advanced

## What's our objective?
http://www.cvxpy.org/

Let's see how we can use optimization to meet a more advanced objective.  We want to both minimize the portfolio variance and also want to closely track a market cap weighted index.  In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.


## Hints

### x vector
To create a vector of M variables $\mathbf{x} = \begin{bmatrix}
x_1 &...& x_M
\end{bmatrix}
$
we can use `cvx.Variable(m)`

### covariance matrix
If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [numpy.cov](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

### portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

Recall that the $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

### Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them.
Recall from the Pythagorean theorem that you can get the distance between two points in an x,y plane by adding the square of the x and y distances and taking the square root.  Extending this to any number of dimensions is called the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

### objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio.  If you choose a higher value for `scale` ($\lambda$), do you think this gives more priority to minimizing the difference, or minimizing the variance?

We can find the objective function using cvxpy `objective = cvx.Minimize()`.  Can you guess what to pass into this function?

### constraints
We can also define our constraints in a list.  For example, you'd want the weights to sum to one. So $\sum_{1}^{n}x = 1$.  You may also need to go long only, which means no shorting, so no negative weights.  So $x_i >0 $ for all $i$. you could save a variable as `[x >= 0, sum(x) == 1]`, where x was created using `cvx.Variable()`.

### optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of $x_A$ and $x_B$ that gave the minimum portfolio variance by using `x.value`

## Quiz

In [13]:
import cvxpy as cvx
import numpy as np

def optimize_portfolio(returns, index_weights, scale=.00001):
    """
    Create a function that takes the return series of a set of stocks, the index weights,
    and scaling factor. The function will minimize a combination of the portfolio variance
    and the distance of its weights from the index weights.  
    The optimization will be constrained to be long only, and the weights should sum to one.
    
    Parameters
    ----------
    returns : numpy.ndarray
        2D array containing stock return series in each row.
        
    index_weights : numpy.ndarray
        1D numpy array containing weights of the index.
        
    scale : float
        The scaling factor applied to the distance between portfolio and index weights
        
    Returns
    -------
    x : np.ndarray
        A numpy ndarray containing the weights of the stocks in the optimized portfolio
    """
    # TODO: Use cvxpy to determine the weights on the assets
    # that minimizes the combination of portfolio variance and distance from index weights
    
    # number of stocks m is number of rows of returns, and also number of index weights
    m = len(returns)
    
    #covariance matrix of returns
    cov = np.cov(returns) 
    
    # x variables (to be found with optimization)
    x = cvx.Variable(m)
    
    #portfolio variance, in quadratic form
    portfolio_variance = cvx.quad_form(x,cov)
    
    # euclidean distance (L2 norm) between portfolio and index weights
    distance_to_index = x - index_weights
    
    #objective function
    objective = cvx.Minimize(portfolio_variance + scale+cvx.norm(distance_to_index, p=2, axis=None))
    
    #constraints
    constraints = [x >= 0, sum(x) == 1]

    #use cvxpy to solve the objective
    problem = cvx.Problem(objective, constraints)
    min_value = problem.solve()
    
    #retrieve the weights of the optimized portfolio
    x_values = x.value
    
    return x_values

quiz_tests_advanced.test_optimize_portfolio(optimize_portfolio)

3
3


AssertionError: Wrong value for optimize_portfolio.

INPUT returns:
[[-0.18576252 -0.01581422  0.00105172 -0.12986104  0.33714221  0.24987429
   0.13473871 -0.49963602 -0.12688536 -0.10273357  0.60654501  0.12984134
  -0.32542494  0.17099126  0.28204022 -0.36595609  0.00647034 -0.09076468
  -0.05990462  0.38059237  0.31038744  0.07550292 -0.37525315  0.25935232
   0.36934776  0.14081369 -0.0241702  -0.33745597  0.4279998  -0.23600535
  -0.43319191 -0.23422814 -0.05238667 -0.14456376 -0.00268175 -0.23770175
  -0.53178952  0.02007463  0.77934214  0.54782649 -0.22074427  0.55586226
  -0.28699057  0.14837687 -0.09203855 -0.11061514 -0.27648705  0.23964429
   0.73277546 -0.05561099 -0.07504271 -0.12607693  0.46138813  0.346017
  -0.08422098 -0.17576238  0.20448621  0.3472651  -0.159198    0.13811693
   0.24694112 -0.12167204  0.5954018   0.29503768  0.36328686  0.09397652
  -0.41613483  0.53448069  0.38932524 -0.17415213  0.05940382  0.21088978
   0.2242418  -0.20895893  0.5780443  -0.02024

In [12]:
"""Test with a 3 simulated stock return series"""
days_per_year = 252
years = 3
total_days = days_per_year * years

return_market = np.random.normal(loc=0.05, scale=0.3, size=days_per_year)
return_1 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
return_2 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
return_3 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
returns = np.array([return_1, return_2, return_3])

"""simulate index weights"""
index_weights = np.array([0.9,0.15,0.05])

"""try out your optimization function"""
x_values = optimize_portfolio(returns, index_weights, scale=.00001)

print(f"The optimized weights are {x_values}, which sum to {sum(x_values):.2f}")

The optimized weights are [0.8666681  0.11666503 0.01666687], which sum to 1.00


If you're feeling stuck, you can check out the solution [here](m3l4_cvxpy_advanced_solution.ipynb)